In [2]:
import urllib.request
import requests
import json
import geojson
import pandas as pd
import plotly.express as px
from datetime import datetime

In [ ]:
#0501013B0 = Amoxicillin

url_0501013B0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501013B0&format=json"
response_0501013B0 = urllib.request.urlopen(url_0501013B0)
data_0501013B0 = json.loads(response_0501013B0.read())
flat_data_0501013B0 = pd.json_normalize(data_0501013B0)
flat_data_0501013B0 = flat_data_0501013B0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501013B0 = flat_data_0501013B0.drop(columns=['items', 'quantity'])
flat_data_0501013B0 .rename(columns={'actual_cost': 'Amoxicillin'}, inplace=True)
flat_data_0501013B0

In [ ]:
#0501030I0 = Doxycycline Hyclate

url_0501030I0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501030I0&format=json"
response_0501030I0 = urllib.request.urlopen(url_0501030I0)
data_0501030I0 = json.loads(response_0501030I0.read())
flat_data_0501030I0 = pd.json_normalize(data_0501030I0)
flat_data_0501030I0 = flat_data_0501030I0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501030I0 = flat_data_0501030I0.drop(columns=['items', 'quantity'])
flat_data_0501030I0.rename(columns={'actual_cost': 'Doxycycline Hyclate'}, inplace=True)
flat_data_0501030I0

In [ ]:
#0501021L0 = Cefalexin

url_0501021L0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501021L0&format=json"
response_0501021L0 = urllib.request.urlopen(url_0501021L0)
data_0501021L0 = json.loads(response_0501021L0.read())
flat_data_0501021L0= pd.json_normalize(data_0501021L0)
flat_data_0501021L0 = flat_data_0501021L0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501021L0 = flat_data_0501021L0.drop(columns=['items', 'quantity'])
flat_data_0501021L0.rename(columns={'actual_cost': 'Cefalexin'}, inplace=True)
flat_data_0501021L0

In [ ]:
join_1 = flat_data_0501013B0.join(flat_data_0501030I0, lsuffix='row_id', rsuffix='row_id')
all_antibiotics_merged = join_1.join(flat_data_0501021L0, lsuffix='row_id', rsuffix='row_id')
all_antibiotics_merged.fillna(0, inplace=True)
all_antibiotics_merged['Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)']= all_antibiotics_merged.iloc[:, -3:].sum(axis=1)
all_antibiotics = all_antibiotics_merged.reset_index()
all_antibiotics.rename(columns={'row_name': 'Clinical Commissioning Group (CCG)', 'row_id': 'CCG code', 'date': 'Date'}, inplace=True)
all_antibiotics

In [ ]:
all_antibiotics_plot = all_antibiotics.groupby(['Date']).sum()
all_antibiotics_plot = all_antibiotics_plot.reset_index()
all_antibiotics_plot = all_antibiotics_plot.round(1)

In [ ]:
pd.options.plotting.backend = "plotly"
fig = px.bar(all_antibiotics_plot, x='Date', y= ["Amoxicillin", "Doxycycline Hyclate", 'Cefalexin'],
color_discrete_sequence=["#003087", "#0072CE", "#41B6E6"],
labels={"value": "Cost (£)", "variable": "Antibiotic"},  
title= "Total cost of Amoxicillin, Doxycycline Hyclate, and Cefalexin (£) per month")
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    autosize=True,
    margin=dict(l=50, r=50, b=50, t=50, pad=4, autoexpand=True),
)
fig.show()

In [ ]:
#CCG pop from NHS digital 
csv_url = "https://files.digital.nhs.uk/40/2232E5/gp-reg-pat-prac-all.csv"
req = requests.get(csv_url)
url_content = req.content
csv_file = open('ccg_pop.csv', 'wb')
csv_file.write(url_content)
csv_file.close()
df1 = pd.read_csv('ccg_pop.csv')
CCG_pop = df1.groupby(['CCG_CODE']).sum().reset_index()
CCG_pop.rename(columns={'CCG_CODE': 'CCG code', 'NUMBER_OF_PATIENTS': 'Number of patients registered at GP practices'}, inplace=True)
CCG_pop

In [ ]:
current_year = datetime.now().year
current_year_str = str(current_year)

In [ ]:
all_antibiotics["Date"] = pd.to_datetime(all_antibiotics["Date"]).apply(lambda x: x.strftime("%Y"))
all_antibiotics_current_year = all_antibiotics.loc[all_antibiotics['Date'] == current_year_str]
df1 = all_antibiotics_current_year.groupby(["CCG code", "Clinical Commissioning Group (CCG)"]).sum()
df2 = df1.drop(columns=['Amoxicillin', 'Doxycycline Hyclate', 'Cefalexin'])
df3 = df2.reset_index()
df4 = df3.join(CCG_pop, rsuffix='CCG code')
df5 = df4.drop(columns=['CCG codeCCG code'])
df5["Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£) per 1000 GP registered patients"] = df5["Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)"]/(df5["Number of patients registered at GP practices"]/1000)
df5.round(2)
df6 = df5.sort_values(by='Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£) per 1000 GP registered patients' , ascending=True)
df6

In [ ]:
pd.options.plotting.backend = "plotly"
fig_2 = px.bar(df6, y='Clinical Commissioning Group (CCG)', x= "Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£) per 1000 GP registered patients",
log_x=True,
labels={"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£) per 1000 GP registered patients": "Cost (£) per 1000 GP registered patients"},
title= "Total cost of Amoxicillin, Doxycycline Hyclate, and Cefalexin (£) per 1000 GP registered patients in %s" %current_year_str, 
color="Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£) per 1000 GP registered patients",
color_continuous_scale=px.colors.sequential.ice,
hover_data={"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£) per 1000 GP registered patients":':.2f'},
width=1500, height=2000)

fig_2.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    autosize=True,
    margin=dict(l=50, r=50, b=50, t=50, pad=4, autoexpand=True),
)
fig_2.show()

In [ ]:
url_ccg_geojson = "https://openprescribing.net/api/1.0/org_location/?org_type=ccg"
response_ccg_geojson = urllib.request.urlopen(url_ccg_geojson)
data_ccg_geojson = geojson.loads(response_ccg_geojson.read())
data_ccg_geojson

In [ ]:
fig_3 = px.choropleth(df6, geojson=data_ccg_geojson, color="Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£) per 1000 GP registered patients",
                    locations="CCG code", featureidkey="properties.code",
                    projection="mercator"
                   )
fig_3.update_geos(fitbounds="locations", visible=False)
fig_3.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_3.show()

In [ ]:
import geopandas

ModuleNotFoundError: No module named 'geopandas'